# 0. Preamble

In [1]:
# External imports
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
from instagrapi.types import StoryLink
import logging

# Internal imports
from utils import import_credentials_from_json, import_dataframe_from_parquet

In [2]:
# Global variables
PATH_TO_CREDENTIALS = '../secrets/credentials.json'
_ = import_credentials_from_json(path=PATH_TO_CREDENTIALS)
IG_USERNAME = _['instagram']['descontosemfim']['username']
IG_PASSWORD = _['instagram']['descontosemfim']['password']
PATH_TO_SESSION_SETTINGS = '../secrets/ig_session.json'

# 1. Logging in to Instagram

## 1.1. The *ig_client_login()* function

In [3]:
# Instagram Client Login function
def ig_client_login(
    username:str='',
    password:str='',
    path_to_session_settings: str = PATH_TO_SESSION_SETTINGS,
) -> Client:

    # Set logging config
    logging.basicConfig(
        filename='../logs/login.txt',
        filemode='a',
        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
        datefmt='%H:%M:%S',
        level=logging.INFO
    )

    # Create Logger object (logging channel)
    logger = logging.getLogger('instathing')
    
    # Create Instagram Client object
    logger.info('Creating Instagram Client object')
    ig_client = Client()
    logger.info('Done')

    # Try to load session settings from file
    try:
        logger.info(f'Loading session settings from {path_to_session_settings}')
        ig_client_settings = ig_client.load_settings(path=path_to_session_settings)
        logger.info('Done')

    # If file not present:
    except FileNotFoundError:
        logger.warning(f'No session settings file found in {path_to_session_settings}')
        
        # Log in to Instagram using username/password
        logger.info('Logging in to Instagram using username/password')
        ig_client.login(username=username, password=password)
        logger.info('Done')
        
    # If file present
    else:
        
        # Apply session settings
        logger.info('Applying session settings to IG Client object')
        ig_client.set_settings(settings=ig_client_settings)
        logger.info('Done')

        # Log in to Instagram
        logger.info('Logging in to Instagram using session settings)')
        ig_client.login(username=username, password=password)
        logger.info('Done')

        # Check session validity (by getting Instagram's news feed)
        try:
            logger.info('Checking session validity')
            ig_client.get_timeline_feed()
            logger.info('Session is valid')
        
        # If expired session
        except LoginRequired:
            logger.warning('Session is expired')
            
            # Clear session settings, keeping uuids
            logger.info('Clearing session settings (keeping uuids)')
            client_settings_old = ig_client.get_settings()
            ig_client.set_settings({})
            ig_client.set_uuids(client_settings_old['uuids'])
            logger.info('Done')
            
            # Log in to Instagram
            logger.info('Logging in to Instagram using username/password')
            ig_client.login(username=username, password=password)
            logger.info('Done')

    # Finally:
    finally:
        
        # Create/Overwrite session settings file
        logger.info(f'Updating {path_to_session_settings}')
        ig_client.dump_settings(path=path_to_session_settings)
        logger.info('Done')
        
        # Set time between requests to 1-3 seconds (mimics human user)
        ig_client.delay_range = [1, 3]
        
        # Return Client object
        return ig_client

## 1.2. Logging in

In [4]:
ig_client = ig_client_login(
    username=IG_USERNAME,
    password=IG_PASSWORD,
)

# 2. Posting Instagram Story

## 2.1. The *post_ig_story()* function

In [5]:
# Post Instagram Story function
def post_ig_story(
    client:Client,
    story_image:str='',
    caption_text:str='',
    link:str='',
    url_btn_x:float=0.5, # 0.0-1.0 (left to right)
    url_btn_y:float=0.5, # 0.0-1.0 (top to bottom)
    url_btn_z:float=0.0,
    url_btn_width:float=0.5, # 0-100% of width (720px)
    url_btn_height:float=0.5, # 0-100% of height (1280px)
    url_btn_rotation:float=0.0
) -> None:

    """
    Post image as Instagram Story with (invisible) marked area for URL button/sticker.
    
    Parameters
    ----------
    client: Client: 
        Instagrapi Client object (with active login).
    
    story_image: str = ''
        Image for posting (720 x 1080 pixels).
    
    caption_text: str = ''
        Caption text.
    
    link: str
        Link to external page. 
        E.g.: 'https://www.google.com'
        
    url_btn_x: float = [0, 1]
        URL button (center) x coordinate, from left to right. 
        E.g.: 0=left, 0.5=center, 1=right.
    
    url_btn_y: float = [0, 1]
        URL button (center) y coordinate, from top to bottom. 
        E.g.: 0=top, 0.5=center, 1=bottom.
    
    url_btn_z: float = 0.0
        ???
    
    url_btn_width: float = [0, 1]
        URL button width (in % of image width). 
        E.g.: 0=0px, 0.5=320, 1=720px.
    
    url_btn_height: float = [0, 1]
        URL button height (in % of image height). 
        E.g.: 0=0px, 0.5=540px, 1=1080px.
    
    url_btn_rotation: float = 0.0
        ???
    
    Returns
    -------
    None
    """
    
    # Upload photo to Instagram Stories
    client.photo_upload_to_story(
        path=story_image,
        caption=caption_text,
        links=[StoryLink(
            webUri=link,
            x=url_btn_x,
            y=url_btn_y,
            z=url_btn_z,
            width=url_btn_width,
            height=url_btn_height,
            rotation=url_btn_rotation
        )],
        extra_data={"audience": "besties"}
    )
    
    # Return None
    return None

## 2.2. Posting an Instagram Story 

In [6]:
post_ig_story(
    client=ig_client,
    story_image='../resources/story_template_720x1280_final.jpg',
    caption_text='',
    link='https://www.netshoes.com.br/',
    url_btn_x=0.5, # Centered (50%)
    url_btn_y=0.8984375, # 1150px / 1280px
    url_btn_z=0.0,
    url_btn_width=0.5833333, # 420px / 720px
    url_btn_height=0.0859375, # 110px / 1280px
    url_btn_rotation=0.0
)

PhotoNotUpload: The password you entered is incorrect. Please try again. If you are sure that the password is correct, then change your IP address, because it is added to the blacklist of the Instagram Server